In [47]:
import os
file_list=[]
labeling_group=['도표사진_윤진 라벨링', '도표사진_혜리 라벨링', '도표사진_경진 라벨링']
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"
for g in labeling_group:
    for l in os.listdir(os.path.join(img_path, g)):
        file_list.append(os.path.join(img_path, g, l))
        
xml_file_list = []
img_file_list = []
for i, j in enumerate(file_list):
    if j.endswith('.xml'):
        xml_file_list.append(j)
        img_file_list.append(file_list[i-1])
        try:
            j[:-3] == file_list[i-1][:-3]
        except: print(j, file_list[i-1])



In [48]:
file_list[0]

'C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\도표사진_윤진 라벨링\\(173호)_월간_ICT산업동향+(최종)_1.png'

In [49]:
import pandas as pd
qa_path =  "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\221207 DIAL 정책지원 AI 데이터 관리.xlsx"
qa_data = pd.read_excel(qa_path, sheet_name='데이터 목록 (도표)')
header = qa_data.iloc[4, 1:8]
qa_data = qa_data.iloc[5:, 1:8]
qa_data.columns = header
qa_data = qa_data.reset_index(drop=True)
qa_data.head()

4,보고서 파일,도표 이름,도표 파일 (보고서 파일_인덱스),작성자,질문,답변 (단위는 제외해주세요!),분류
0,‘청년주택’을 넘어 ‘20대주택’으로,〈그림 2〉 2009년~2018년간서울시 구별20~29세 인구 순유입,‘청년주택’을 넘어 ‘20대주택’으로_1,최민규,2009년~2018년간서울시 구별20~29세 인구 순유입이 가장 많은 구는 어디인가?,관악구,막대
1,‘청년주택’을 넘어 ‘20대주택’으로,〈그림 3〉 2009년~2018년간서울시 대학지역20~29세 인구의 시도간유입 추이,‘청년주택’을 넘어 ‘20대주택’으로_2,최민규,2018년 서울시 대학지역20~29세 인구의 시도간유입이 15000명 이하인 지역은...,마포-서대문,꺾은선
2,‘청년주택’을 넘어 ‘20대주택’으로,〈그림 4〉 20~29세 인구의 서울 전입 순이동 추이,‘청년주택’을 넘어 ‘20대주택’으로_3,최민규,25~29세 인구의 서울 전입 순이동이 가장 적었던 년도는 몇년인가?,2015년,꺾은선
3,(2차웨이브) 2020년 장애인고용패널조사(2차웨이브 5차조사),〔그림1-4-2〕조사진행추이,(2차웨이브) 2020년 장애인고용패널조사(2차웨이브 5차조사)_1,최민규,7주차의 5차조사 진행추이는 몇퍼센트 인가?,68.5,꺾은선
4,(21-10호)_월간_ICT산업동향,그림1-1전체반도체시장전망및메모리반도체고정거래가격추이,(21-10호)_월간_ICT산업동향_1,최민규,21.1에서의 D램 DDR4 8GB의 고정거래가격은?,3.00,꺾은선


In [50]:
qa_data.columns

Index(['보고서 파일', '도표 이름', '도표 파일  (보고서 파일_인덱스)', '작성자', '질문',
       '답변 (단위는 제외해주세요!)', '분류'],
      dtype='object', name=4)

In [51]:
import cv2
import xmltodict
import json 
import pickle 

plot_list = []
qid = 1000000
for index, (img_file, xml_file) in enumerate(zip(img_file_list, xml_file_list)):
    img_file = img_file.split('\\')[-1]
    with open(xml_file, 'rt', encoding='UTF-8') as f:
        xmlString = f.read()
    
    anno_data_list = xmltodict.parse(xmlString)['annotation']['object']
    bboxes = []
    
    for anno_data in anno_data_list:
        coordinates = list(map(int, list(anno_data['bndbox'].values()))) 
        if '\\' in anno_data['name']: 
            print(xml_file.split('\\')[-2:])
            break
        plot_cls = anno_data['name'].rsplit('==', 1)
        # if plot_cls[0] =='digital grade restrictiveness index': print(xml_file.split('\\')[-2:])
        if len(plot_cls) == 2:
            bboxes.append((plot_cls[0], (coordinates), plot_cls[1]))
        elif len(plot_cls) == 1:
            bboxes.append((plot_cls[0], (coordinates), None))
    try:
        is_line = False
        for bb in bboxes:
            if 'line' in bb[0]: is_line = True
        if is_line:
            plot_list.append(
                {
                    "qid": qid,
                    "question": list(qa_data.loc[qa_data['도표 파일  (보고서 파일_인덱스)'] == img_file[:-4]]['질문'])[0],
                    "answer": list(qa_data.loc[qa_data['도표 파일  (보고서 파일_인덱스)'] == img_file[:-4]]['답변 (단위는 제외해주세요!)'])[0],
                    "image": img_file,
                    "bboxes": bboxes,
                }
            )
    except:
        # print("error")
        continue

    qid += 1
    # if index > 10 : break

with open('data/h_line_annotation.pkl',"wb") as f:
    pickle.dump(plot_list, f)

In [52]:

with open('data/h_line_annotation.pkl', 'rb') as handle:
    output = pickle.load(handle)
    print(len(output), output)

46 [{'qid': 1000004, 'question': '2011년 농촌관광 만족도는 몇점인가?', 'answer': '78.3', 'image': '(2016-01)농촌 마을관광 활성화 방안(최종)_3.png', 'bboxes': [('x_tick', [124, 331, 184, 356], '2009년'), ('x_tick', [282, 332, 345, 354], '2011년'), ('x_tick', [438, 330, 502, 358], '2012년'), ('x_tick', [598, 335, 660, 358], '2014년'), ('val', [134, 215, 173, 236], '75.2'), ('val', [296, 161, 329, 183], '78.3'), ('val', [448, 282, 486, 303], '72.0'), ('val', [614, 204, 646, 226], '76.1'), ('val', [132, 106, 167, 123], '79.2'), ('val', [289, 56, 327, 78], '81.6'), ('val', [449, 83, 486, 100], '80.4'), ('val', [607, 79, 642, 98], '80.6'), ('t_legend', [484, 34, 577, 54], '농촌관광 만족도'), ('t_legend', [613, 35, 706, 55], '국내관광 만족도'), ('v_legend', [460, 39, 486, 47], None), ('v_legend', [591, 37, 612, 48], None), ('title', [683, 5, 748, 25], '(단위: 점)'), ('line_1', [157, 81, 312, 132], None), ('line_1', [313, 79, 470, 109], None), ('line_1', [470, 100, 632, 108], None), ('line_0', [152, 150, 315, 213], None), ('line_0', [312, 

In [53]:
# import os
# from shutil import copyfile

# def mkdirs(newdir,mode=777):
#     try:
#         os.makedirs(newdir, mode)
#     except OSError as err:
#         return err

# file_list=[]
# labeling_group=['도표사진_윤진 라벨링', '도표사진_혜리 라벨링', '도표사진_경진 라벨링']
# img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"

# # mkdirs(os.path.join(img_path, 'human'))

# for g in labeling_group:
#     for l in os.listdir(os.path.join(img_path, g)):
#         file_list.append(os.path.join(img_path, g, l))
        
# xml_file_list = []
# img_file_list = []
# for i, j in enumerate(file_list):
#     if j.endswith('.png'):
#         # print(j)
#         copyfile(j , os.path.join(img_path, 'human', j.split('\\')[-1]))



In [54]:
import os
file_list=[]
labeling_group=['샘플 라벨링']
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"
for g in labeling_group:
    for l in os.listdir(os.path.join(img_path, g)):
        file_list.append(os.path.join(img_path, g, l))
xml_file_list = []
img_file_list = []
for i, j in enumerate(file_list):
    if j.endswith('.xml'):
        xml_file_list.append(j)
        img_file_list.append(file_list[i-1])
        try:
            j[:-3] == file_list[i-1][:-3]
        except: print(j, file_list[i-1])



In [55]:
qa_data = [
    {
        "problem_no": "10",
        "task_no": "3",
        "ref_file": "agc_asdf2hg436tgwdf_figure_10.png",
        "question": "장학 및 컨설팅 경험이 있는 어린이집의 비율은 얼마인가?",
        "answer": "26.8" 
    },
    {
        "problem_no": "34",
        "task_no": "3",
        "ref_file": "agc_asdf2hg436tgwdf_figure_34.png",
        "question": "전통문양 콘텐츠 활용 패션 디자인 분야 중 가장 큰 비중을 차지하는 분야는 무엇인가?",
        "answer": "의류"
    },
    {
        "problem_no": "36",
        "task_no": "3",
        "ref_file": "agc_asdf2hg436tgwdf_figure_36.png",
        "question": "문화포털 전통문양 아카이브 만족도 조사에서 경제성 항목에서 만족이라고 응답한 비율은 얼마인가?",
        "answer": "73.8"
    },
    {
        "problem_no": "39",
        "task_no": "3",
        "ref_file": "AGC_p9aDe43fE2_figure_39.png",
        "question": "문화포털 전통문양 아카이브에 대한 비이용 이유로 콘텐츠 품질과 관련하여 응답한 비율은 얼마인가?",
        "answer": "15.0"
    },
    {
        "problem_no": "40",
        "task_no": "3",
        "ref_file": "AGC_p9aDe43fE2_figure_40.png",
        "question": "2022년 2분기를 기준으로 가격이 가장 낮은 곡물은 무엇인가?",
        "answer": "쌀"
    }
]

In [56]:
import cv2
import xmltodict
import json 
import pickle 

plot_list = []
qid = 990000
for index, (img_file, xml_file) in enumerate(zip(img_file_list, xml_file_list)):
    img_file = img_file.split('\\')[-1]
    with open(xml_file, 'rt', encoding='UTF-8') as f:
        xmlString = f.read()
    
    anno_data_list = xmltodict.parse(xmlString)['annotation']['object']
    bboxes = []
    
    for anno_data in anno_data_list:
        coordinates = list(map(int, list(anno_data['bndbox'].values()))) 
        if '\\' in anno_data['name']: 
            print(xml_file.split('\\')[-2:])
            break
        plot_cls = anno_data['name'].rsplit('==', 1)
        # if plot_cls[0] =='digital grade restrictiveness index': print(xml_file.split('\\')[-2:])
        if len(plot_cls) == 2:
            bboxes.append((plot_cls[0], (coordinates), plot_cls[1]))
        elif len(plot_cls) == 1:
            bboxes.append((plot_cls[0], (coordinates), None))
    try:
        is_line = False
        for bb in bboxes:
            if 'line' in bb[0]: is_line = True
        if is_line:
            for i, j in enumerate(qa_data):
                if j["ref_file"]==img_file:
                    question = qa_data[i]['question']
                    answer = qa_data[i]['answer']
            plot_list.append(
                {
                    "qid": qid,
                    "question": question,
                    "answer": answer,
                    "image": img_file,
                    "bboxes": bboxes,
                }
            )
    except:
        # print("error")
        continue

    qid += 1
    # if index > 10 : break

with open('data/s_line_annotation.pkl',"wb") as f:
    pickle.dump(plot_list, f)

In [57]:

with open('data/s_line_annotation.pkl', 'rb') as handle:
    output = pickle.load(handle)
    print(len(output), output)

2 [{'qid': 990004, 'question': '문화포털 전통문양 아카이브 만족도 조사에서 경제성 항목에서 만족이라고 응답한 비율은 얼마인가?', 'answer': '73.8', 'image': 'agc_asdf2hg436tgwdf_figure_36.png', 'bboxes': [('val', [20, 118, 45, 131], '76.8'), ('val', [19, 51, 45, 64], '87.6'), ('val', [74, 167, 97, 180], '55.4'), ('val', [85, 57, 114, 73], '86.9'), ('val', [140, 198, 166, 210], '41.1'), ('val', [144, 70, 172, 83], '82.3'), ('val', [202, 154, 229, 167], '60.7'), ('val', [206, 58, 233, 73], '85.1'), ('val', [267, 88, 294, 102], '72.7'), ('val', [260, 200, 285, 214], '39.3'), ('val', [329, 71, 355, 85], '80.1'), ('val', [324, 208, 350, 223], '37.5'), ('val', [390, 84, 420, 97], '76.6'), ('val', [385, 235, 410, 249], '30.4'), ('val', [457, 158, 484, 171], '40.8'), ('val', [454, 273, 476, 287], '10.7'), ('val', [509, 105, 538, 120], '66.0'), ('val', [518, 248, 542, 261], '23.2'), ('val', [571, 88, 601, 104], '73.8'), ('val', [577, 181, 601, 194], '50.0'), ('val', [635, 92, 664, 106], '70.9'), ('val', [641, 241, 667, 255], '25.0'), ('

In [58]:
# import os
# from shutil import copyfile

# def mkdirs(newdir,mode=777):
#     try:
#         os.makedirs(newdir, mode)
#     except OSError as err:
#         return err

# file_list=[]
# labeling_group=['샘플 라벨링']
# img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"

# mkdirs(os.path.join(img_path, 'sample'))

# for g in labeling_group:
#     for l in os.listdir(os.path.join(img_path, g)):
#         file_list.append(os.path.join(img_path, g, l))
        
# xml_file_list = []
# img_file_list = []
# for i, j in enumerate(file_list):
#     if j.endswith('.png'):
#         # print(j)
#         copyfile(j , os.path.join(img_path, 'sample', j.split('\\')[-1]))

